In [ ]:
!pip install praw
import praw 
import urllib.request
import os 
from PIL import Image
import json 
from tqdm.auto import tqdm
import requests
from meme_ocr.memeocr import MemeOCR        
import re


     |████████████████████████████████| 176 kB 5.0 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 


In [ ]:
"""
Image downloader
"""
def downloader(link,path):
    # f= urllib.request.urlretrieve(link, path)
    # print(f)
    # print(f"downloading {link}")
    # return f
    try:
        img= requests.get(link)
        if img.status_code!=200:
            return
        # print(img)
        img_data = img.content
        with open(path, 'wb') as handler:
            handler.write(img_data)
        return True
    except:
      print("download failed")
      return False


In [ ]:
"""
PRAW instance to scrape reddit
"""
reddit = praw.Reddit(
    client_id="dQG4qfGiEo-KjQV8wd11jQ",
    client_secret="5oekxFnVLknPdwSwIHTWBSdkbzD45Q",
    user_agent="my user agent",
    check_for_async=False
)

In [ ]:
"""SUBREDDIT LIST"""
subreddit_list = ['meme','memes','Memes_Of_The_Dank','darkmemers','MemeYourEnthusiasm','dankmemes','Grimdank','MemesIRL','dankmeme','okbuddyretard',"terriblefacebookmemes","ComedyCemetery"]

In [ ]:
PATH  = "/content/drive/MyDrive/CSSdata2"
if not os.path.exists(PATH):
    os.makedirs(PATH)

In [ ]:
import pandas as pd
import requests
import json
import time
"""
Function to read data using Pushshift
"""
def getPushshiftData(after, sub,searchq=None):
    if searchq!=None:
        url = 'https://api.pushshift.io/reddit/search/submission?&size=500&after='+str(after)+'&subreddit='+str(sub)+'&q='+str(searchq)
    else:
        url = 'https://api.pushshift.io/reddit/search/submission?&size=500&after='+str(after)+'&subreddit='+str(sub)
    time.sleep(0.7)
    r = requests.get(url)
    if len(r.text)>5 and r.status_code==200:
        data = json.loads(r.text)
        return data['data']
    else:
        print("failed",r.status_code,r.text)
        return []

In [ ]:
import pandas as pd
import requests
import json
import time

def getPushshiftData(after, sub,searchq=None):
    if searchq!=None:
        url = 'https://api.pushshift.io/reddit/search/submission?&size=500&after='+str(after)+'&subreddit='+str(sub)+'&q='+str(searchq)
    else:
        url = 'https://api.pushshift.io/reddit/search/submission?&size=1000&after='+str(after)+'&subreddit='+str(sub)
    time.sleep(0.5)
    r = requests.get(url)
    if len(r.text)>5 and r.status_code==200:
        data = json.loads(r.text)
        return data['data']
    else:
        return []
"""
Downloading all jsons
"""
for sub in subreddit_list:
    #list of post ID's
    posts = []
 
    print(f"processing subreddit {sub}")
    after = "1647789753"
    sixmonths = '1637520835'
    ayear = "1619031835"
    twoyear = "1588003650"
    searchq='women | woman '
    data = getPushshiftData(twoyear, sub,searchq=searchq)
    print(data)
    # Will run until all posts have been gathered 
    # from the 'after' date up until todays date
    while len(data) > 0 and len(posts)<25000:
        for submission in data:
            posts.append(submission)
        # Calls getPushshiftData() with the created date of the last submission
        print(f"Scraped {len(posts)}")
        data = getPushshiftData(sub=sub, after=data[-1]['created_utc'],searchq=searchq)


    obj = {}
    # obj['sub'] = sub
    obj[sub] = posts



    # Save to json for later use
    with open(f"{PATH}/{sub}~{searchq}.json", "w") as jsonFile:
        json.dump(obj, jsonFile)

processing subreddit ComedyCemetery
[{'all_awardings': [], 'allow_live_comments': False, 'author': 'iam4real', 'author_flair_css_class': None, 'author_flair_richtext': [], 'author_flair_text': None, 'author_flair_type': 'text', 'author_fullname': 't2_4ocu7', 'author_patreon_flair': False, 'author_premium': True, 'awarders': [], 'can_mod_post': False, 'contest_mode': False, 'created_utc': 1588192508, 'domain': 'i.imgur.com', 'full_link': 'https://www.reddit.com/r/ComedyCemetery/comments/gah4kj/environmentalism_and_women_bad/', 'gildings': {}, 'id': 'gah4kj', 'is_crosspostable': True, 'is_meta': False, 'is_original_content': False, 'is_reddit_media_domain': False, 'is_robot_indexable': True, 'is_self': False, 'is_video': False, 'link_flair_background_color': '', 'link_flair_richtext': [], 'link_flair_text_color': 'dark', 'link_flair_type': 'text', 'locked': False, 'media_only': False, 'no_follow': True, 'num_comments': 2, 'num_crossposts': 0, 'over_18': False, 'parent_whitelist_status': 

In [ ]:
"""
Downloading all images for jsons
"""
for jfile in os.listdir(os.path.join(PATH,'sub_files_filtered')):
    json_path = os.path.join(PATH,"sub_files_filtered",jfile)
    all_posts = json.load(open(json_path,'r'))
    sub_name = jfile.split("~")[0]
    print(f"processing subreddit {sub_name}")
    save_path = os.path.join(PATH,sub_name+"~filtered")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    progress_bar = tqdm(range(len(all_posts[sub_name])))
    for submission in all_posts[sub_name]:
        data = {}
        if submission['is_video'] == True or submission['is_self']==True:
            continue
        if 'post_hint' in submission and 'video' in submission['post_hint']:
            continue

        try:
            link = submission['url']
            # print(link)
            # print(data['link'])
            if 'jpg' in link:
                file_name = os.path.join(save_path,submission['id'])
            else:
                continue
            stat = downloader(link,file_name+".jpg")
            if stat==False:
                continue
            if not os.path.exists(file_name+".jpg"):
                continue
            demo = Image.open(file_name+".jpg").convert("RGB")
        except KeyboardInterrupt:
            continue

        with open(file_name+".json","w") as f:
             json.dump(submission,f)
        progress_bar.update(1) 
    #     break
    

processing subreddit terriblefacebookmemes


  0%|          | 0/342 [00:00<?, ?it/s]

processing subreddit ComedyCemetery


  0%|          | 0/190 [00:00<?, ?it/s]

In [ ]:
"""##add ocr text to images"""
ocr = MemeOCR()
postfix = "~filtered"
for subreddit in  ["terriblefacebookmemes","ComedyCemetery"]:
    folder_path = os.path.join(PATH,subreddit+postfix)
    if not os.path.isdir(folder_path):
        continue
    total_files = os.listdir(folder_path)
    total_files = [i for i in total_files if "jpg" in i]
    print(f"processing subreddit {subreddit}")
    progress_bar = tqdm(range(len(total_files)))
    c = 0
    for f in total_files:
        meme_path = os.path.join(folder_path,f)
        meme_json_path = meme_path.split(".")[0]+".json"
        if not os.path.exists(meme_json_path):
            continue
        meme_json = json.load(open(meme_json_path,"r"))
        text = ocr.recognize(meme_path)
        text = " ".join(text)
        text  = re.sub(r'[^A-Za-z0-9 ]+', '', text)
        if len(text)>0:
            meme_json["ocr"] = text
        else:
            meme_json["ocr"] = ""
        with open(meme_json_path,"w") as f:
            json.dump(meme_json,f)
        
        progress_bar.update(1)
        c+=1
        # if c>1000:
        #     break
        

processing subreddit terriblefacebookmemes


  0%|          | 0/223 [00:00<?, ?it/s]

processing subreddit ComedyCemetery


  0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
"""collect upvote information from posts"""
for sub in subreddit_list:
    #list of post ID's
    posts = []
    #Subreddit to query
    # sub='okbuddyretard'
    # Unix timestamp of date to crawl from.
    # 2022/03/20
    print(f"processing subreddit {sub}")
    after = "1647789753"
    sixmonths = '1637520835'
    ayear = "1619031835"
    twoyear = "1588003650"
    data = getPushshiftData(twoyear, sub)
    try:
      current = int(ayear)
      while len(data)>0:
          for submission in data:
              id = submission["id"]
              submission["upvotes"] = reddit.submission(id=id).score
              posts.append(submission)
          current += 2592000
          print(f"Scraped {len(posts)}")
          data = getPushshiftData(sub=sub, after=current)
    except:
      pass
      
    obj = {}
    # obj['sub'] = sub
    obj[sub] = posts



    # Save to json for later use
    with open(f"{PATH}/{sub}~overall.json", "w") as jsonFile:
        json.dump(obj, jsonFile)
#didnt process meme your enthusisasm 

processing subreddit MemesIRL
Scraped 100
Scraped 200
Scraped 300
Scraped 400
Scraped 500
Scraped 600
Scraped 700
Scraped 800
Scraped 900
Scraped 1000
processing subreddit dankmeme
Scraped 95
Scraped 194
Scraped 292
Scraped 389
Scraped 489
Scraped 589
Scraped 689
Scraped 789
Scraped 889
Scraped 989
Scraped 1089
Scraped 1179
Scraped 1231
processing subreddit okbuddyretard
Scraped 100
Scraped 200
Scraped 299
Scraped 399
Scraped 499
Scraped 599
Scraped 699
Scraped 799
Scraped 899
Scraped 999
Scraped 1099
Scraped 1199
Scraped 1299
processing subreddit terriblefacebookmemes
Scraped 100
Scraped 200
Scraped 300
Scraped 400
Scraped 500
Scraped 600
Scraped 700
Scraped 800
Scraped 900
Scraped 1000
Scraped 1100
Scraped 1200
Scraped 1300
processing subreddit ComedyCemetery
Scraped 99
Scraped 199
Scraped 299
Scraped 399
Scraped 499
Scraped 599
Scraped 699
Scraped 799
Scraped 899
Scraped 999
Scraped 1099
Scraped 1199
Scraped 1299
